In [13]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import hilbert
from antropy import sample_entropy
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import glob
from collections import Counter
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier



In [4]:
dataf = "feature_full_fixed.csv"
df = pd.read_csv(dataf)
epohe_labels = df["Sleep_Stage"].tolist()

list_feature = df.drop(columns=["Sleep_Stage"]).values.tolist()
list_feature = [row[1:] for row in list_feature]


In [5]:
arr = np.array(list_feature)
arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)

arr = np.clip(arr, -1e6, 1e6)


In [6]:
def balance_classes(X, y):
    valid_mask = y != 'Missing'
    X = X[valid_mask]
    y = y[valid_mask]
    unique_classes = np.unique(y)
    min_class_count = min(Counter(y).values())  # smallest class size
    print(min_class_count)
    X_balanced = []
    y_balanced = []

    for cls in unique_classes:

        # Get all samples for this class
        X_cls = X[y == cls]
        y_cls = y[y == cls]

        # Downsample to match smallest class
        X_down, y_down = resample(X_cls, y_cls, 
                                  replace=False,
                                  n_samples=min_class_count,
                                  random_state=42)

        X_balanced.append(X_down)
        y_balanced.append(y_down)

    # Concatenate all balanced classes
    X_final = np.vstack(X_balanced)
    y_final = np.concatenate(y_balanced)

    return X_final, y_final 

In [7]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(arr)

X = torch.tensor(scaled_features, dtype=torch.float32)
input_size = X.shape[1]

labels_raw = df["Sleep_Stage"]

labels_cat = pd.Categorical(labels_raw)

y = torch.tensor(labels_cat.codes, dtype=torch.long)

X_bal, y_bal = balance_classes(X.numpy(), y.numpy()) #BALANSIRANE - manje izvrsenje rada
#X_bal, y_bal = X, y #NEIZBALANSIRANE - testiraj da ne dodje do problema
X_bal = torch.tensor(X_bal, dtype=torch.float32)
y_bal = torch.tensor(y_bal, dtype=torch.long)

8112


KNN ALGORITAM

In [18]:
clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
clf.fit(X_bal, y_bal) 

y_pred = clf.predict(X_bal)

print(classification_report(y_bal, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      8112
           1       1.00      0.99      0.99      8112
           2       1.00      0.97      0.98      8112
           3       1.00      0.99      0.99      8112
           4       1.00      0.98      0.99      8112

    accuracy                           0.99     40560
   macro avg       0.99      0.99      0.99     40560
weighted avg       0.99      0.99      0.99     40560



RANDOM FOREST ALGORITAM


In [16]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_bal, y_bal) 

y_pred = clf.predict(X_bal)

print(classification_report(y_bal, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      8112
           1       1.00      0.99      0.99      8112
           2       0.95      1.00      0.97      8112
           3       1.00      0.99      0.99      8112
           4       1.00      0.98      0.99      8112

    accuracy                           0.99     40560
   macro avg       0.99      0.99      0.99     40560
weighted avg       0.99      0.99      0.99     40560



Neural Network - ne radi normalno

In [17]:
clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_bal, y_bal) 

y_pred = clf.predict(X_bal)

print(classification_report(y_bal, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.11      0.16      8112
           1       0.42      0.43      0.43      8112
           2       0.76      0.84      0.80      8112
           3       0.45      0.61      0.52      8112
           4       0.53      0.65      0.59      8112

    accuracy                           0.53     40560
   macro avg       0.50      0.53      0.50     40560
weighted avg       0.50      0.53      0.50     40560



C:\Users\lazar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
